# This Simulated Annealing Is Designed for Teaching Purposes

The main goal of this simulated annealing framework is to help everyone learn how to tune parameters and how to improve their own annealing strategies.

First of all, temperature selection is critical and must be tested based on your specific problem. Different temperature settings can lead to improvements under different conditions.
    For the initial temperature, if you observe significant early improvements, it can be set relatively high, for example in the range of 8–16. When progress slows down in later stages, it is better to gradually reduce the initial temperature, such as to 3–1. The ending temperature is generally not recommended to be changed frequently, although you can still experiment to find a more suitable value if needed.

Second, regarding how to improve your simulated annealing, you should first analyze what problems exist in your current implementation. In my experience, many annealing implementations are generated by AI, and AI often makes a critical mistake: it sets the movement or rotation step sizes too large. However, in the later stages of optimization, each object (e.g., each tree) is already positioned within a very small range. Large perturbations at this stage cause the algorithm to waste a lot of time exploring irrelevant or unproductive regions.

Third, if you are unsure about how to choose appropriate movement or rotation parameters, you can ask AI to add adaptive step-size mechanisms. This approach can significantly reduce the time spent on manual parameter tuning. However, its limitation is that convergence is usually slower, so it should be used with this trade-off in mind.

In [1]:
%%writefile sa.py
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
import time
import multiprocessing
import math
import random
import os
from collections import defaultdict
import argparse

getcontext().prec = 50
scale_factor = Decimal('1e18')

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0', item_id=None):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)
        self.item_id = item_id
        self.polygon = self._create_polygon()

    def _create_polygon(self):
        trunk_w = Decimal('0.15'); trunk_h = Decimal('0.2')
        base_w = Decimal('0.7'); base_y = Decimal('0.0')
        mid_w = Decimal('0.4'); tier_2_y = Decimal('0.25')
        top_w = Decimal('0.25'); tier_1_y = Decimal('0.5')
        tip_y = Decimal('0.8'); trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        return affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor)
        )

    def clone(self) -> "ChristmasTree":
        new_tree = ChristmasTree.__new__(ChristmasTree)
        new_tree.center_x = self.center_x
        new_tree.center_y = self.center_y
        new_tree.angle = self.angle
        new_tree.item_id = self.item_id
        new_tree.polygon = self.polygon
        return new_tree


def splitmix64(x: int) -> int:
    """Deterministic 64-bit mixer (same spirit as C++ SplitMix64)."""
    x &= 0xFFFFFFFFFFFFFFFF
    x = (x + 0x9E3779B97F4A7C15) & 0xFFFFFFFFFFFFFFFF
    z = x
    z = (z ^ (z >> 30)) * 0xBF58476D1CE4E5B9 & 0xFFFFFFFFFFFFFFFF
    z = (z ^ (z >> 27)) * 0x94D049BB133111EB & 0xFFFFFFFFFFFFFFFF
    z = z ^ (z >> 31)
    return z & 0xFFFFFFFFFFFFFFFF


def parse_int_range(s: str):
    """Parse '40-80' (inclusive) or single int like '12'. Returns (min,max) or None."""
    if s is None:
        return None
    s = str(s).strip()
    if not s:
        return None
    if '-' in s:
        a, b = s.split('-', 1)
        a = int(a.strip())
        b = int(b.strip())
        if a > b:
            a, b = b, a
        return a, b
    v = int(s)
    return v, v


def get_tree_list_side_length_fast(polygons) -> float:
    if not polygons:
        return 0.0
    minx, miny, maxx, maxy = polygons[0].bounds
    for p in polygons[1:]:
        b = p.bounds
        if b[0] < minx: minx = b[0]
        if b[1] < miny: miny = b[1]
        if b[2] > maxx: maxx = b[2]
        if b[3] > maxy: maxy = b[3]
    return max(maxx - minx, maxy - miny) / float(scale_factor)


def validate_no_overlaps(polygons):
    if not polygons:
        return True

    strtree = STRtree(polygons)

    for i, poly in enumerate(polygons):
        candidates = strtree.query(poly)

        for cand in candidates:
            if hasattr(cand, "geom_type"):
                other = cand
                if other is poly:
                    continue
            else:
                j = int(cand)
                if j == i:
                    continue
                other = polygons[j]

            if (not poly.disjoint(other)) and (not poly.touches(other)):
                return False

    return True


def parse_csv(csv_path):
    print(f'Loading csv: {csv_path}')
    result = pd.read_csv(csv_path)
    for col in ['x', 'y', 'deg']:
        if result[col].dtype == object:
            result[col] = result[col].astype(str).str.strip('s')

    # id is usually like "<group>_<item>"; keep item_id so we can preserve IDs on save.
    result[['group_id', 'item_id']] = result['id'].str.split('_', n=1, expand=True)

    dict_of_tree_list = {}
    for group_id, group_data in result.groupby('group_id'):
        # iterrows -> itertuples
        tree_list = [
            ChristmasTree(center_x=str(row.x), center_y=str(row.y), angle=str(row.deg), item_id=str(row.item_id))
            for row in group_data.itertuples(index=False)
        ]
        dict_of_tree_list[group_id] = tree_list
    return dict_of_tree_list


def save_dict_to_csv(dict_of_tree_list, output_path):
    print(f"Saving solution to {output_path}...")
    data = []
    sorted_keys = sorted(dict_of_tree_list.keys(), key=lambda x: int(x))
    for group_id in sorted_keys:
        trees = dict_of_tree_list[group_id]
        for i, tree in enumerate(trees):
            item_id = tree.item_id if tree.item_id is not None else str(i)
            data.append({
                'id': f"{group_id}_{item_id}",
                'x': f"s{tree.center_x}",
                'y': f"s{tree.center_y}",
                'deg': f"s{tree.angle}",
            })
    df = pd.DataFrame(data)[['id', 'x', 'y', 'deg']]
    df.to_csv(output_path, index=False)
    print("Save complete.")



def run_simulated_annealing(args):
    (
        group_id,
        initial_trees,
        max_iterations,
        t_start,
        t_end,
        base_seed,
        adapt_window,
        target_accept,
        adapt_strength,
        t_min_multiplier,
        t_max_multiplier,
        stagnation_iters,
        reheat_fraction,
        max_reheats,
    ) = args
    n_trees = len(initial_trees)

    gid_int = int(group_id)
    task_seed = splitmix64((int(base_seed) ^ (gid_int * 0x9E3779B97F4A7C15)) & 0xFFFFFFFFFFFFFFFF)
    rng = random.Random(task_seed)

    is_small_n = n_trees <= 50

    if is_small_n:
        effective_max_iter = max_iterations * 3
        effective_t_start = t_start * 2.0
        gravity_weight = 1e-4
    else:
        effective_max_iter = max_iterations
        effective_t_start = t_start
        gravity_weight = 1e-6

    state = []
    for t in initial_trees:
        cx_float = float(t.center_x) * float(scale_factor)
        cy_float = float(t.center_y) * float(scale_factor)
        state.append({
            'poly': t.polygon,
            'cx': cx_float,
            'cy': cy_float,
            'angle': float(t.angle),
        })

    current_polys = [s['poly'] for s in state]
    current_bounds = [p.bounds for p in current_polys]

    scale_f = float(scale_factor)
    inv_scale_f = 1.0 / scale_f
    inv_scale_f2 = 1.0 / (scale_f * scale_f)

    def _envelope_from_bounds(bounds_list):
        if not bounds_list:
            return (0.0, 0.0, 0.0, 0.0)
        minx, miny, maxx, maxy = bounds_list[0]
        for b in bounds_list[1:]:
            if b[0] < minx: minx = b[0]
            if b[1] < miny: miny = b[1]
            if b[2] > maxx: maxx = b[2]
            if b[3] > maxy: maxy = b[3]
        return (minx, miny, maxx, maxy)

    def _envelope_from_bounds_replace(bounds_list, replace_i: int, replace_bounds):
        if not bounds_list:
            return (0.0, 0.0, 0.0, 0.0)
        b0 = replace_bounds if replace_i == 0 else bounds_list[0]
        minx, miny, maxx, maxy = b0
        for i, b in enumerate(bounds_list[1:], start=1):
            if i == replace_i:
                b = replace_bounds
            if b[0] < minx: minx = b[0]
            if b[1] < miny: miny = b[1]
            if b[2] > maxx: maxx = b[2]
            if b[3] > maxy: maxy = b[3]
        return (minx, miny, maxx, maxy)

    def _side_len_from_env(env):
        minx, miny, maxx, maxy = env
        return max(maxx - minx, maxy - miny) * inv_scale_f


    env = _envelope_from_bounds(current_bounds)
    dist_sum = 0.0
    for s in state:
        dist_sum += s['cx'] * s['cx'] + s['cy'] * s['cy']

    # Dynamic boundary penalty: keep a moving limit_size that tightens
    # when we find a new best feasible side.
    boundary_penalty_coeff = 1000.0
    tighten_factor = 0.995

    def boundary_penalty(env_local, replace_i=None, replace_bounds=None, limit_size_unscaled=0.0):
        if not (limit_size_unscaled > 0.0):
            return 0.0
        # Penalize any tree bbox that exceeds the square [minx, minx+limit] x [miny, miny+limit].
        env_minx, env_miny, _, _ = env_local
        x_limit_scaled = env_minx + limit_size_unscaled * scale_f
        y_limit_scaled = env_miny + limit_size_unscaled * scale_f
        penalty = 0.0
        for i in range(n_trees):
            b = replace_bounds if (replace_bounds is not None and i == replace_i) else current_bounds[i]
            # Only penalize overflow on +x/+y, matching the C++ change request.
            dx = b[2] - x_limit_scaled
            if dx > 0.0:
                penalty += boundary_penalty_coeff * (dx * dx) * inv_scale_f2
            dy = b[3] - y_limit_scaled
            if dy > 0.0:
                penalty += boundary_penalty_coeff * (dy * dy) * inv_scale_f2
        return penalty

    def energy_from(env_local, dist_sum_local, progress01, replace_i=None, replace_bounds=None, limit_size_unscaled=0.0):
        side_len = _side_len_from_env(env_local)
        normalized_dist = (dist_sum_local * inv_scale_f2) / max(1, n_trees)
        grav_w = gravity_weight * max(0.0, 1.0 - progress01)
        penalty = boundary_penalty(env_local, replace_i, replace_bounds, limit_size_unscaled)
        # Match C++: side^2 + gravity + penalty
        return (side_len * side_len) + grav_w * normalized_dist + penalty, side_len

    limit_size = _side_len_from_env(env)
    current_energy, current_side_len = energy_from(env, dist_sum, 0.0, None, None, limit_size)

    best_state_params = [{'cx': s['cx'], 'cy': s['cy'], 'angle': s['angle']} for s in state]
    best_real_score = current_side_len

    # Temperature schedule (kept for acceptance probability) with adaptive multiplier + clamps.
    schedule_T = effective_t_start
    cooling_rate = math.pow(t_end / effective_t_start, 1.0 / effective_max_iter)
    T_mult = 1.0
    T_abs_min = max(0.0, t_end * max(1.0, float(t_min_multiplier)))
    T_abs_max = max(T_abs_min, effective_t_start * max(1.0, float(t_max_multiplier)))

    def clamp_T(x):
        if x < T_abs_min:
            return T_abs_min
        if x > T_abs_max:
            return T_abs_max
        return x

    T = clamp_T(schedule_T * T_mult)

    # Adaptive temperature control (acceptance-rate targeting)
    adapt_window = max(0, int(adapt_window))
    try:
        target_accept = float(target_accept)
    except Exception:
        target_accept = 0.25
    if not (0.0 < target_accept < 1.0):
        target_accept = 0.25
    try:
        adapt_strength = float(adapt_strength)
    except Exception:
        adapt_strength = 0.0
    if not (adapt_strength >= 0.0):
        adapt_strength = 0.0
    window_trials = 0
    window_accepts = 0

    # Stagnation reheating
    stagnation_iters = max(0, int(stagnation_iters))
    try:
        reheat_fraction = float(reheat_fraction)
    except Exception:
        reheat_fraction = 0.0
    if not (reheat_fraction >= 0.0):
        reheat_fraction = 0.0
    max_reheats = int(max_reheats)
    reheats_done = 0
    last_progress_it = 0  # last time best_real_score improved OR we reheated

    # Adaptive step-size control (decoupled from temperature)
    class StepController:
        def __init__(self):
            self.move_scale = 1.0
            self.rotate_scale = 1.0
            self.attempts = 0
            self.accepts = 0

    step_ctrl = StepController()
    STEP_BATCH = 500
    STEP_TARGET = 0.25
    STEP_MIN = 1e-5
    STEP_MAX = 10.0

    def clamp_step(x):
        if x < STEP_MIN:
            return STEP_MIN
        if x > STEP_MAX:
            return STEP_MAX
        return x

    def step_update(accepted: bool):
        step_ctrl.attempts += 1
        if accepted:
            step_ctrl.accepts += 1
        if step_ctrl.attempts >= STEP_BATCH:
            acc = step_ctrl.accepts / max(1, step_ctrl.attempts)
            if acc > STEP_TARGET:
                step_ctrl.move_scale = clamp_step(step_ctrl.move_scale * 1.02)
                step_ctrl.rotate_scale = clamp_step(step_ctrl.rotate_scale * 1.02)
            elif acc < STEP_TARGET:
                step_ctrl.move_scale = clamp_step(step_ctrl.move_scale * 0.98)
                step_ctrl.rotate_scale = clamp_step(step_ctrl.rotate_scale * 0.98)
            step_ctrl.attempts = 0
            step_ctrl.accepts = 0

    for i in range(effective_max_iter):
        progress = i / effective_max_iter

        # Stagnation-triggered reheating (based on best improvements)
        if stagnation_iters > 0 and (i - last_progress_it) >= stagnation_iters:
            if max_reheats == 0 or reheats_done < max_reheats:
                target_T = effective_t_start * reheat_fraction
                if target_T > 0.0:
                    denom = max(1e-30, schedule_T)
                    T_mult = max(T_mult, target_T / denom)
                    T = clamp_T(schedule_T * T_mult)
                    reheats_done += 1
                    # Reset temp adaptation window so it reflects post-reheat dynamics.
                    window_trials = 0
                    window_accepts = 0
                last_progress_it = i

        # Adaptive step sizes (decoupled from T)
        base_move = 0.005 if is_small_n else 0.001
        base_rotate = 0.001 if is_small_n else 0.002
        move_scale = base_move * step_ctrl.move_scale
        rotate_scale = base_rotate * step_ctrl.rotate_scale

        idx = rng.randint(0, n_trees - 1)
        target = state[idx]

        orig_poly = target['poly']
        orig_bounds = current_bounds[idx]
        orig_cx, orig_cy, orig_angle = target['cx'], target['cy'], target['angle']

        dx = (rng.random() - 0.5) * scale_f * 0.1 * move_scale
        dy = (rng.random() - 0.5) * scale_f * 0.1 * move_scale
        d_angle = (rng.random() - 0.5) * rotate_scale

        rotated_poly = affinity.rotate(orig_poly, d_angle, origin=(orig_cx, orig_cy))
        new_poly = affinity.translate(rotated_poly, xoff=dx, yoff=dy)
        new_bounds = new_poly.bounds
        minx, miny, maxx, maxy = new_bounds

        new_cx = orig_cx + dx
        new_cy = orig_cy + dy
        new_angle = orig_angle + d_angle

        collision = False
        for k in range(n_trees):
            if k == idx:
                continue
            ox1, oy1, ox2, oy2 = current_bounds[k]
            if maxx < ox1 or minx > ox2 or maxy < oy1 or miny > oy2:
                continue
            other = current_polys[k]

            if (not new_poly.disjoint(other)) and (not new_poly.touches(other)):
                collision = True
                break

        if collision:
            # Count as an attempt for the step controller.
            step_update(False)

            schedule_T *= cooling_rate
            T = clamp_T(schedule_T * T_mult)
            continue

        old_d = orig_cx * orig_cx + orig_cy * orig_cy
        new_d = new_cx * new_cx + new_cy * new_cy
        cand_dist_sum = dist_sum - old_d + new_d

        env_minx, env_miny, env_maxx, env_maxy = env
        need_recompute = (
            (orig_bounds[0] == env_minx and new_bounds[0] > env_minx) or
            (orig_bounds[1] == env_miny and new_bounds[1] > env_miny) or
            (orig_bounds[2] == env_maxx and new_bounds[2] < env_maxx) or
            (orig_bounds[3] == env_maxy and new_bounds[3] < env_maxy)
        )
        if need_recompute:
            cand_env = _envelope_from_bounds_replace(current_bounds, idx, new_bounds)
        else:
            cand_env = (
                min(env_minx, new_bounds[0]),
                min(env_miny, new_bounds[1]),
                max(env_maxx, new_bounds[2]),
                max(env_maxy, new_bounds[3]),
            )

        new_energy, new_real_score = energy_from(
            cand_env,
            cand_dist_sum,
            progress,
            replace_i=idx,
            replace_bounds=new_bounds,
            limit_size_unscaled=limit_size,
        )
        delta = new_energy - current_energy

        accept = False
        if delta < 0:
            accept = True
        else:
            if T > 1e-12:
                scale = 1000.0 / max(1.0, 2.0 * current_side_len)
                prob = math.exp(-delta * scale / T)
                accept = rng.random() < prob

        # Update adaptive temperature stats only for feasible (non-colliding) proposals.
        if adapt_window > 0:
            window_trials += 1
            if accept:
                window_accepts += 1
            if window_trials >= adapt_window:
                acc = window_accepts / max(1, window_trials)
                adj = math.exp(adapt_strength * (target_accept - acc))
                adj = max(0.5, min(2.0, adj))
                T_mult *= adj
                T = clamp_T(schedule_T * T_mult)
                window_trials = 0
                window_accepts = 0

        if accept:
            current_polys[idx] = new_poly
            current_bounds[idx] = new_bounds
            target['poly'] = new_poly
            target['cx'] = new_cx
            target['cy'] = new_cy
            target['angle'] = new_angle

            current_energy = new_energy
            env = cand_env
            dist_sum = cand_dist_sum
            current_side_len = new_real_score

            if new_real_score < best_real_score:
                best_real_score = new_real_score
                last_progress_it = i
                # Dynamic compression: tighten boundary after finding a new best feasible solution.
                limit_size = best_real_score * tighten_factor
                for k in range(n_trees):
                    best_state_params[k]['cx'] = state[k]['cx']
                    best_state_params[k]['cy'] = state[k]['cy']
                    best_state_params[k]['angle'] = state[k]['angle']

        # Step controller feedback uses accept/reject outcome of this iteration.
        step_update(accept)

        schedule_T *= cooling_rate
        T = clamp_T(schedule_T * T_mult)

    final_trees = []
    final_polys_check = []
    for p in best_state_params:
        cx_dec = Decimal(p['cx']) / scale_factor
        cy_dec = Decimal(p['cy']) / scale_factor
        angle_dec = Decimal(p['angle'])
        new_t = ChristmasTree(str(cx_dec), str(cy_dec), str(angle_dec))
        final_trees.append(new_t)
        final_polys_check.append(new_t.polygon)

    if not validate_no_overlaps(final_polys_check):
        orig_score = get_tree_list_side_length_fast([t.polygon for t in initial_trees])
        return group_id, initial_trees, orig_score

    return group_id, final_trees, best_real_score

def main():
    parser = argparse.ArgumentParser(description="Santa-2025 SA optimizer (Python/Shapely).")
    parser.add_argument("--input", default="/kaggle/input/a-bit-better-the-best-open-source-result-for-date/submission.csv", help="Input CSV path")
    parser.add_argument("--output", default="/kaggle/working/submission.csv", help="Output CSV path")
    parser.add_argument("--iter", type=int, default=10000, help="Base iterations per group")
    parser.add_argument("--tstart", type=float, default=1.0, help="Start temperature")
    parser.add_argument("--tend", type=float, default=0.00003, help="End temperature")
    parser.add_argument("--processes", default="auto", help="Process count or 'auto'")
    parser.add_argument("--seed", type=int, default=4489708, help="Deterministic base seed")
    parser.add_argument("--range", default=None, help="Only optimize groups in inclusive range a-b")
    parser.add_argument("--gid_min", type=int, default=None, help="Only optimize groups >= gid_min")
    parser.add_argument("--gid_max", type=int, default=None, help="Only optimize groups <= gid_max")
    parser.add_argument("--time_limit_sec", type=int, default=11.5 * 3600, help="Wall time limit")
    parser.add_argument("--save_every", type=int, default=20, help="Checkpoint frequency by finished groups")

    # adaptive temperature + reheating
    parser.add_argument("--adapt_window", type=int, default=200, help="Adaptive temperature window")
    parser.add_argument("--target_accept", type=float, default=0.25, help="Target accept rate for adaptive temperature")
    parser.add_argument("--adapt_strength", type=float, default=0.75, help="Strength of adaptive temperature updates")
    parser.add_argument("--t_min_multiplier", type=float, default=1.0, help="Min T multiplier relative to t_end")
    parser.add_argument("--t_max_multiplier", type=float, default=2.0, help="Max T multiplier relative to t_start")
    parser.add_argument("--stagnation", type=int, default=20, help="Stagnation iters before reheating")
    parser.add_argument("--reheat_fraction", type=float, default=0.8, help="Reheat target as fraction of t_start")
    parser.add_argument("--max_reheats", type=int, default=20, help="0 => unlimited reheats")
    args = parser.parse_args()

    INPUT_CSV = args.input
    OUTPUT_CSV = args.output

    try:
        dict_of_tree_list = parse_csv(INPUT_CSV)
    except FileNotFoundError:
        print(f"Error: Could not find {INPUT_CSV}.")
        return

    all_groups_sorted = sorted(dict_of_tree_list.keys(), key=lambda x: int(x), reverse=True)

    gid_min = args.gid_min
    gid_max = args.gid_max
    r = parse_int_range(args.range)
    if r is not None:
        gid_min, gid_max = r

    if gid_min is None:
        gid_min = -10**18
    if gid_max is None:
        gid_max = 10**18

    groups_to_optimize = [gid for gid in all_groups_sorted if gid_min <= int(gid) <= gid_max]

    MAX_ITER = int(args.iter)
    T_START = float(args.tstart)
    T_END = float(args.tend)

    KAGGLE_TIME_LIMIT_SEC = int(args.time_limit_sec)
    SAVE_EVERY_N_GROUPS = int(args.save_every)

    tasks = []
    for gid in groups_to_optimize:
        tasks.append(
            (
                gid,
                dict_of_tree_list[gid],
                MAX_ITER,
                T_START,
                T_END,
                args.seed,
                args.adapt_window,
                args.target_accept,
                args.adapt_strength,
                args.t_min_multiplier,
                args.t_max_multiplier,
                args.stagnation,
                args.reheat_fraction,
                args.max_reheats,
            )
        )

    if str(args.processes).lower() == "auto":
        num_processes = multiprocessing.cpu_count()
    else:
        num_processes = max(1, int(args.processes))

    # Don't spawn more workers than tasks.
    num_processes = min(num_processes, max(1, len(tasks)))

    print(f"Starting SA on {len(tasks)}/{len(all_groups_sorted)} groups using {num_processes} processes...")
    if gid_min != -10**18 or gid_max != 10**18:
        print(f"Group filter: {gid_min}-{gid_max} (inclusive)")
    print(f"Seed(base): {args.seed}")
    print(f"Time Limit: {KAGGLE_TIME_LIMIT_SEC / 3600:.2f} hours")
    print("Press Ctrl+C to stop early and save progress.")

    start_time = time.time()
    improved_count = 0
    total_tasks = len(tasks)
    finished_tasks = 0

    pool = multiprocessing.Pool(processes=num_processes)

    try:
        results_iter = pool.imap_unordered(run_simulated_annealing, tasks, chunksize=1)

        for result in results_iter:
            group_id, optimized_trees, score = result
            finished_tasks += 1

            orig_polys = [t.polygon for t in dict_of_tree_list[group_id]]
            orig_score = get_tree_list_side_length_fast(orig_polys)

            status_msg = ""
            if score < orig_score:
                diff = orig_score - score
                if diff > 1e-12:
                    status_msg = f" -> Improved! (-{diff:.6f})"
                    dict_of_tree_list[group_id] = optimized_trees
                    improved_count += 1

            elapsed_time = time.time() - start_time
            if elapsed_time > KAGGLE_TIME_LIMIT_SEC:
                print(
                    f"\n[WARNING] Time limit approach ({elapsed_time / 3600:.2f}h). "
                    "Stopping early to save data safely."
                )
                pool.terminate()
                break

            if finished_tasks % SAVE_EVERY_N_GROUPS == 0:
                print(
                    f"   >>> Auto-saving checkpoint at "
                    f"{finished_tasks}/{total_tasks}..."
                )
                save_dict_to_csv(dict_of_tree_list, OUTPUT_CSV)

            print(
                f"[{finished_tasks}/{total_tasks}] "
                f"G:{group_id} {orig_score:.5f}->{score:.5f} {status_msg}"
            )

        pool.close()
        pool.join()
        print(f"\nOptimization finished normally in {time.time() - start_time:.2f}s")

    except KeyboardInterrupt:
        print("\n\n!!! Caught Ctrl+C (KeyboardInterrupt) !!!")
        print("Terminating workers and saving current progress...")
        pool.terminate()
        pool.join()
    except Exception as e:
        print(f"\nAn error occurred: {e}")
        pool.terminate()
        pool.join()
    finally:
        print(f"Final Save. Total Improved: {improved_count}")
        save_dict_to_csv(dict_of_tree_list, OUTPUT_CSV)


if __name__ == '__main__':
    multiprocessing.freeze_support()
    main()


Writing sa.py


In [2]:
!python sa.py \
  --input /kaggle/input/a-bit-better-the-best-open-source-result-for-date/submission.csv \
  --output /kaggle/working/submission.csv \
  --iter 1000000 \
  --tstart 3.0 \
  --tend 0.0003 \
  --processes auto \
  --seed 4489708 \
  --time_limit_sec 41400 \
  --save_every 20 \
  --adapt_window 200 \
  --target_accept 0.25 \
  --adapt_strength 0.75 \
  --t_min_multiplier 1.0 \
  --t_max_multiplier 2.0 \
  --stagnation 20 \
  --reheat_fraction 0.8 \
  --max_reheats 20


Loading csv: /kaggle/input/a-bit-better-the-best-open-source-result-for-date/submission.csv
Starting SA on 200/200 groups using 4 processes...
Seed(base): 4489708
Time Limit: 11.50 hours
Press Ctrl+C to stop early and save progress.
[1/200] G:197 8.19702->8.19702 
[2/200] G:198 8.22139->8.22139 
[3/200] G:199 8.22699->8.22699 
[4/200] G:200 8.23155->8.23155 
[5/200] G:194 8.12370->8.12370 
[6/200] G:195 8.14175->8.14175 
[7/200] G:193 8.10250->8.10250 
[8/200] G:196 8.16540->8.16540 
[9/200] G:192 8.02389->8.02389 
[10/200] G:191 8.02033->8.02033 
[11/200] G:190 8.01661->8.01661 
[12/200] G:189 8.00776->8.00776 
[13/200] G:188 7.99471->7.99471 
[14/200] G:187 7.98135->7.98135 
[15/200] G:185 7.94518->7.94518 
[16/200] G:186 7.96985->7.96985 
[17/200] G:183 7.89263->7.89263 
[18/200] G:182 7.74984->7.74984 
[19/200] G:181 7.72866->7.72866 
   >>> Auto-saving checkpoint at 20/200...
Saving solution to /kaggle/working/submission.csv...
Save complete.
[20/200] G:184 7.93232->7.93232 
[21/2